In [8]:
from torch.utils.data import Dataset
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.1.0'

### 引入模块 读取数据

PyTorch 通过 torch.utils.data 对数据集进行加载，对常用数据集 torchvision 进行预先的实现。

In [9]:
#定义一个数据集
class BulldozerDataset(Dataset):
    """ 数据集演示 """
    def __init__(self, csv_file):
        """实现初始化方法，在初始化的时候将数据读载入"""
        self.df=pd.read_csv(csv_file)
    def __len__(self):
        '''
        返回df的长度
        '''
        return len(self.df)
    def __getitem__(self, idx):
        '''
        根据 idx 返回一行数据
        '''
        return self.df.iloc[idx].SalePrice

In [1]:
ds_demo= BulldozerDataset('median_benchmark.csv')

'1.1.0'

### 构建计算图（构建网络模型）

    class torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
    卷积运算后，张量维度变化计算公式：
$$\frac{n-f+2p}{s}+1$$

In [3]:

class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
       
        self.conv1 = nn.Conv2d(1, 6, 3) 
        
        self.fc1   = nn.Linear(1350, 10) 
   
    def forward(self, x): 
        print(x.size()) 
 
        x = self.conv1(x) 
        x = F.relu(x)# shape 【1,6,30,30】
        print(x.size()) 
        x = F.max_pool2d(x, (2, 2)) 
        x = F.relu(x)
        print(x.size())


        x = x.view(x.size()[0], -1) 
        print(x.size()) 
        x = self.fc1(x)        
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


In [11]:

input = torch.randn(1, 1, 32, 32) 
out = net(input)
out.size()
net.zero_grad() 
out.backward(torch.ones(1,10)) # 反向传播的实现是PyTorch自动实现的，我们只要调用这个函数即可

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


### 损失函数与优化器

在深度学习中要用到各种各样的损失函数（loss function），这些损失函数可看作是一种特殊的layer，PyTorch也将这些损失函数实现为nn.Module的子类。然而在实际使用中通常将这些loss function专门提取出来，和主模型互相独立。如交叉熵，均方误差。

In [13]:
#交叉熵
# batch_size=3，计算对应每个类别的分数（只有两个类别）
score = torch.randn(3, 2)
# 三个样本分别属于1，0，1类，label必须是LongTensor
label = torch.Tensor([1, 0, 1]).long()

# loss与普通的layer无差异
criterion = nn.CrossEntropyLoss()
loss = criterion(score, label)
loss

tensor(0.7123)

In [14]:
#均方误差

y = torch.arange(0,10).view(1,10).float()
criterion = nn.MSELoss()
loss = criterion(out, y)
#loss是个scalar，我们可以直接用item获取到他的python类型的数值
print(loss.item())

30.05908966064453


在反向传播计算完所有参数的梯度后，还需要使用优化方法来更新网络的权重和参数，例如随机梯度下降法、RMSProp、Adam等

In [15]:
import torch.optim
out = net(input) # 这里调用的时候会打印出我们在forword函数中打印的x的大小
criterion = nn.MSELoss()
loss = criterion(out, y)
#新建一个优化器，SGD只需要要调整的参数和学习率
optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)
# 先梯度清零(与net.zero_grad()效果一样)
optimizer.zero_grad() 
loss.backward()

#更新参数
optimizer.step()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


<generator object Module.parameters at 0x0000000002929200>